In [1]:
import os.path
from collections import Counter

import numpy as np


data_dir = "data"
X_test = np.load(os.path.join(data_dir, "X_test.npy"))
y_test = np.load(os.path.join(data_dir, "y_test.npy"))
person_test = np.load(os.path.join(data_dir, "person_test.npy")).squeeze(axis=1)
X_train_valid = np.load(os.path.join(data_dir, "X_train_valid.npy"))
y_train_valid = np.load(os.path.join(data_dir, "y_train_valid.npy"))
person_train_valid = np.load(
    os.path.join(data_dir, "person_train_valid.npy")
).squeeze(axis=1)

n_class = len(set(y_train_valid))
min_y = min(y_train_valid)
y_train_valid = y_train_valid - min_y
y_test = y_test - min_y

In [2]:
print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'. format (X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))

Training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115,)
Person test shape: (443,)


In [3]:
import random

import torch


seed = 1
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

device = torch.device("mps")
device

device(type='mps')

In [4]:
from models import *
from trainer import *

n_trials = 5

In [5]:
s1_indices_train = [i for i, s in enumerate(person_train_valid) if s == 1]
s1_indices_test = [i for i, s in enumerate(person_test) if s == 1]
X_train_valid_s1 = X_train_valid[s1_indices_train]
y_train_valid_s1 = y_train_valid[s1_indices_train]
X_test_s1 = X_test[s1_indices_test]
y_test_s1 = y_test[s1_indices_test]

# make test set dataloader
# Convert data to tensors
X_tensor = torch.FloatTensor(X_test_s1)
y_tensor = torch.LongTensor(y_test_s1)

# Combine X and y into a TensorDataset
dataset = TensorDataset(X_tensor, y_tensor)

# Prepare dataloaders
test_dataloader = DataLoader(dataset, batch_size=256, shuffle=False)

In [6]:
valid_accs = []
test_accs = []

for i in range(n_trials):
    cnn = CNN(input_size=X_train_valid_s1.shape[1:], N=n_class).to(device)
    valid_acc = fit(
        cnn, X_train_valid_s1, y_train_valid_s1, device, epochs=50, random_state=i, print_acc=True
    )

    test_acc, _ = evaluate(cnn, test_dataloader, device)
    valid_accs.append(valid_acc)
    test_accs.append(test_acc)

round(np.mean(valid_accs), 5), round(np.mean(test_accs), 5)

  0%|          | 0/50 [00:00<?, ?it/s]

train_acc=0.25, valid_acc=0.22917
train_acc=0.32447, valid_acc=0.22917
train_acc=0.44149, valid_acc=0.22917
train_acc=0.5266, valid_acc=0.22917
train_acc=0.60638, valid_acc=0.27083
train_acc=0.63298, valid_acc=0.27083
train_acc=0.67021, valid_acc=0.25
train_acc=0.78723, valid_acc=0.22917
train_acc=0.80319, valid_acc=0.25
train_acc=0.80319, valid_acc=0.25
train_acc=0.82447, valid_acc=0.25
train_acc=0.83511, valid_acc=0.27083
train_acc=0.85638, valid_acc=0.25
train_acc=0.89894, valid_acc=0.27083
train_acc=0.90957, valid_acc=0.25
train_acc=0.90957, valid_acc=0.25
train_acc=0.8883, valid_acc=0.27083
train_acc=0.91489, valid_acc=0.27083
train_acc=0.91489, valid_acc=0.27083
train_acc=0.93085, valid_acc=0.27083
train_acc=0.96809, valid_acc=0.27083
train_acc=0.94681, valid_acc=0.27083
train_acc=0.94149, valid_acc=0.27083
train_acc=0.96809, valid_acc=0.27083
train_acc=0.95213, valid_acc=0.25
train_acc=0.95213, valid_acc=0.27083
train_acc=0.96277, valid_acc=0.27083
train_acc=0.95745, valid_acc=0

ValueError: too many values to unpack (expected 2)